<a href="https://colab.research.google.com/github/ShabnaIlmi/Data-Science-Group-Project/blob/Future_Risk_prediction/DSGP_startover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load datasets
file1 = "/mnt/data/recipes_nodup.csv"
file2 = "/mnt/data/chem.csv"

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Display first few rows
print("📌 recipes_nodup.csv")
print(df1.head(), "\n\n")

print("📌 chem.csv")
print(df2.head(), "\n\n")

# Check for missing values
print("🔍 Missing values in recipes_nodup:\n", df1.isnull().sum(), "\n")
print("🔍 Missing values in chem:\n", df2.isnull().sum(), "\n")


In [ ]:
# Drop rows with too many missing values
df1 = df1.dropna()
df2 = df2.dropna()

print(f"📉 recipes_nodup: {df1.shape}, chem: {df2.shape}")  # Check new sizes


df1 = df1.drop_duplicates()
df2 = df2.drop_duplicates()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=500)  # Limit to top 500 words
df1_tfidf = vectorizer.fit_transform(df1["Chemical Names"]).toarray()

# Convert to DataFrame
df1_tfidf = pd.DataFrame(df1_tfidf, columns=vectorizer.get_feature_names_out())

# Merge back to dataset
df1 = pd.concat([df1, df1_tfidf], axis=1).drop(columns=["Chemical Names"])


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df1[["Quantities"]] = scaler.fit_transform(df1[["Quantities"]])


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df1["Risk Level Encoded"] = label_encoder.fit_transform(df1["Risk Level"])


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df1["Risk Level Encoded"] = label_encoder.fit_transform(df1["Risk Level"])


In [ ]:
df1.to_csv("/mnt/data/processed_data.csv", index=False)
print("✅ Processed data saved as processed_data.csv")
